In [2]:
%load_ext watermark

In [3]:
%watermark

Last updated: 2024-01-12T07:49:47.755245-05:00

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.18.1

Compiler    : GCC 13.2.0
OS          : Linux
Release     : 6.5.0-14-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit



In [4]:
%watermark --gpu

GPU Info: 
  GPU 0: Quadro RTX 5000



In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import sklearn
import gc
import os
import xgboost as xgb

In [6]:
%watermark --iversions

xgboost: 2.0.3
pandas : 2.1.4
sklearn: 1.3.2
numpy  : 1.26.3



In [7]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [8]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [9]:
optuna_log = pd.read_csv('optuna_xgb_output_0.csv')
optuna_log.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.260580,0.518387,0.588129,logloss,0.803620,0.001024,19,73,binary:logistic,0.734509,gpu_hist,COMPLETE
1,1,0.272333,0.554771,0.386440,logloss,0.515043,0.004684,19,226,binary:logistic,0.740431,gpu_hist,COMPLETE
2,2,0.273310,0.001337,0.321878,logloss,0.224638,0.035482,25,235,binary:logistic,0.647692,gpu_hist,COMPLETE
3,3,0.213742,0.475032,0.907356,logloss,0.025882,0.019162,21,5,binary:logistic,0.939027,gpu_hist,COMPLETE
4,4,0.250605,7.983068,0.797204,logloss,1.452732,0.037789,8,12,binary:logistic,0.532211,gpu_hist,COMPLETE


In [10]:
optuna_log.value.max()

0.2834224878819652

In [11]:
max_value_row = optuna_log[optuna_log['value'] == optuna_log['value'].max()]

In [12]:
max_value_row

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
94,94,0.283422,0.654147,0.917583,logloss,4.645511,0.013001,7,194,binary:logistic,0.66007,gpu_hist,COMPLETE


In [13]:
columns = test.columns[1:]

In [14]:
X = train[columns].values
X_test = test[columns].values
Y = train.target.values

In [15]:
params = {'objective': 'binary:logistic',
 'tree_method': 'hist',
 'lambda': 4.645511,
 'alpha': 0.654147,
 'colsample_bytree': 0.917,
 'subsample': 0.66,
 'learning_rate': 0.013,
 'max_depth': 7,
 'min_child_weight': 194,
 'eval_metric': 'logloss'}

In [16]:
dtest = xgb.DMatrix(X_test, enable_categorical=True)

In [17]:
%%time
train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape
num_round = 1000

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    dtrain = xgb.DMatrix(train_features, train_target, enable_categorical=True)
    dval = xgb.DMatrix(val_features, val_target, enable_categorical=True)
    
    model = xgb.train(params, dtrain, num_round)
    model.set_param({'predictor': 'cpu_predictor'})
    val_pred = model.predict(dval)
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict(dtest)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [07:50:30] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Fold normalized: 0.2859987739508921
Fitting fold 2


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [07:50:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Fold normalized: 0.2926920190271769
Fitting fold 3


...

Fold normalized: 0.2752018416077811
Fitting fold 4


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [07:51:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Fold normalized: 0.2839655533013239
Fitting fold 5


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [07:51:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Fold normalized: 0.2792386205373169
CPU times: user 20min 14s, sys: 1.15 s, total: 20min 15s
Wall time: 1min 18s


In [18]:
gini_normalized(Y, train_oof)

0.28320824744370143

In [23]:
%%time
params = {'objective': 'binary:logistic',
 'tree_method': 'hist',
 'device':'cuda',
 'lambda': 4.645511,
 'alpha': 0.654147,
 'colsample_bytree': 0.917,
 'subsample': 0.66,
 'learning_rate': 0.013,
 'max_depth': 7,
 'min_child_weight': 194,
 'eval_metric': 'logloss'}

train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape
num_round = 1000

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    dtrain = xgb.DMatrix(train_features, train_target, enable_categorical=True)
    dval = xgb.DMatrix(val_features, val_target, enable_categorical=True)
    
    model = xgb.train(params, dtrain, num_round)
    #model.set_param({'predictor': 'gpu_predictor'})
    val_pred = model.predict(dval)
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict(dtest)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold normalized: 0.2867144764531161
Fitting fold 2
Fold normalized: 0.2926881219215813
Fitting fold 3
Fold normalized: 0.2741990239593723
Fitting fold 4
Fold normalized: 0.2830241574984101
Fitting fold 5
Fold normalized: 0.280211134158975
CPU times: user 42.6 s, sys: 429 ms, total: 43 s
Wall time: 33.5 s


In [24]:
gini_normalized(Y, train_oof)

0.28317274980925405

In [25]:
roc_auc_score(Y, train_oof)

0.6415863758691099

In [26]:
sample_submission['target'] = test_preds

In [27]:
sample_submission.to_csv('../submissions/XGB_5_fold_optuna_best_2.csv.zip', index=False, compression='zip')

In [28]:
val_features = X[val_index]
val_target = Y[val_index]

In [29]:
dval = xgb.DMatrix(val_features, val_target, enable_categorical=True)

On the leaderbaord this submission scores 0.2791 on the public test set, and 0.28408on the private dataset.

In [30]:
%%time
shap_values = model.predict(dval, pred_contribs=True)

CPU times: user 2.96 s, sys: 32 ms, total: 2.99 s
Wall time: 2.98 s


In [32]:
shap_values.shape

(119042, 58)

In [37]:
np.abs(shap_values).sum(axis=0)

array([3.9728125e+03, 1.7090957e+03, 1.2026953e+04, 4.5702261e+03,
       9.7621963e+03, 4.6826323e+03, 6.0037432e+03, 2.3123777e+03,
       2.4973862e+03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.1130110e+04, 4.3606924e+03,
       8.2153809e+03, 1.8910826e+02, 6.6007964e+03, 5.2044297e+03,
       8.9063994e+03, 5.7245239e+03, 1.2235306e+03, 5.4801782e+03,
       6.4713391e+02, 6.2480658e+02, 1.7466602e+03, 5.0587578e+03,
       3.6994308e+02, 3.4698804e+03, 0.0000000e+00, 2.7114497e+03,
       2.5779067e+03, 1.0256482e+03, 1.9154455e+04, 3.1489202e+03,
       2.7862456e+03, 9.6365448e+02, 1.9221951e+03, 1.0894573e+03,
       7.1927997e+02, 6.0091980e+02, 1.0261910e+03, 9.4148303e+02,
       9.6176831e+02, 9.4298242e+02, 1.1642261e+03, 1.4763823e+03,
       8.4133698e+02, 7.9220209e+02, 1.4086056e+03, 2.8659177e+02,
       2.8195444e+02, 6.2611981e+02, 2.9945767e+02, 7.2081982e+02,
       6.7193085e+02, 3.9066959e+05], dtype=float32)